# Loading data from preprocessed pickle file

In [1]:
import pickle

# open the pickle file in read-binary mode
with open('..//model-data/preprocessing/train-val-sets/train_val_split.pkl', 'rb') as f:
    data = pickle.load(f)

# storing data
images_train = data['images_train']
images_val = data['images_val']
labels_train = data['labels_train']
labels_val = data['labels_val']

# Creating model

In [ ]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Rescaling(scale=1.0/255, input_shape=(256, 256, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

In [2]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Rescaling(scale=1.0/255, input_shape=(256, 256, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

2025-01-25 18:44:54.127910: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 18:44:54.251242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-25 18:44:54.320049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-25 18:44:54.320857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 18:44:54.425641: I tensorflow/core/platform/cpu_feature_gua

# Compling the model

In [3]:
# compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fitting the model

In [4]:
# sliding window model training (to handle batching due to tensorflow gpu mem errors)
for w in range(2):
    # calculating a third of the data size
    third_of_data = len(images_train) // 3
    HR_size = third_of_data
    LR_size = 0
    
    for i in range(3):
        #print(LR_size, HR_size)
        images_train_third = images_train[LR_size:HR_size]
        labels_train_third = labels_train[LR_size:HR_size]
    
        model.fit(images_train_third, labels_train_third, epochs=1)
        
        LR_size += third_of_data;
        HR_size += third_of_data

I0000 00:00:1737830705.251654     639 service.cc:145] XLA service 0x7f108c004c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737830705.252612     639 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-01-25 18:45:05.321193: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-25 18:45:05.556828: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1737830706.464755     693 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_765', 544 bytes spill stores, 544 bytes spill loads



  4/440 ━━━━━━━━━━━━━━━━━━━━ 20s 46ms/step - accuracy: 0.0879 - loss: 61.3693  

I0000 00:00:1737830710.565854     639 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


440/440 ━━━━━━━━━━━━━━━━━━━━ 25s 42ms/step - accuracy: 0.4911 - loss: 50.0276
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.7247 - loss: 11.7103
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.7854 - loss: 6.3240
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.8419 - loss: 3.5697
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.8596 - loss: 2.6103
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 42ms/step - accuracy: 0.8855 - loss: 1.7514


# Validating the model

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score

# converting one hot encoded labels to class indices
real_labels = np.argmax(labels_val, axis=1)

# predict on the validation set
predictions = model.predict(images_val)

# converting predictions from one hot encoded probabiliti|es to class indices
predicted_labels = np.argmax(predictions, axis=1)\

# comparing predictions with the real labels
print("Validation Accuracy:", accuracy_score(real_labels, predicted_labels))

331/331 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Validation Accuracy: 0.7517280560552978


# [TESTING]

In [ ]:
import numpy as np

def one_hot_max(arr):
    result = np.zeros_like(arr)
    result[np.argmax(arr)] = 1
    return result

print(one_hot_max(predictions[4]))

In [ ]:
print(labels_val[4])

# Saving the model

In [ ]:
import os

save_dir = './model-saves'
os.makedirs(save_dir, exist_ok=True)

model.save_weights(f'{save_dir}/80_percent_accuracy.weights.h5')